In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
#! SETUP 1 - DO NOT CHANGE, MOVE NOR COPY
import sys, os
_snlp_book_dir = "../../../../../"
sys.path.append(_snlp_book_dir)
# docker image contains tensorflow 0.10.0rc0. We will support execution of only that version!
import statnlpbook.nn as nn
import tensorflow as tf
import numpy as np

In [2]:
#! SETUP 2 - DO NOT CHANGE, MOVE NOR COPY
data_path = _snlp_book_dir + "data/nn/"
data_train = nn.load_corpus(data_path + "train.tsv")
data_train = nn.load_corpus(data_path + "train.tsv")
data_dev = nn.load_corpus(data_path + "dev.tsv")
assert(len(data_train) == 45502)

In [3]:
# data_train = data_train[40000:]
# len(data_train)

In [4]:
def loadWord2Vec(filename):
    vocab = {}
    embd = []
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(' ')
        for i,token in enumerate(row):
            if i == 0:
                vocab[row[i]] = len(vocab)
            else:
                embd.append(float(row[i]))
    print('Successfully load.')
    file.close()    
    return vocab,embd

In [5]:
import re
def tokenize(input):
    token = re.compile("[\w]+(?=n't)|n't|'s|\'m|\'ll|[\w]+|[.?!;,:]")
    tokens = token.findall(input)
    return tokens

In [6]:
def pipeline2(data,vocab=None,embd=None,max_sent_len_=None):
    is_ext_vocab = True
    if vocab is None and embd is None: 
        is_ext_vocab = False
        #load word2vect
        filename = 'word2vec.txt'
        vocab, vec = loadWord2Vec(filename)
        embd = np.array(vec,dtype=np.float32)
        embd = np.reshape(embd,(len(vocab),50))
    
    max_sent_len = -1
    data_sentences = []
    data_orders = []
    for instance in data:
        sents = []
        for sentence in instance['story']:
            sent = []
            tokenized = tokenize(sentence)
            for token in tokenized:
                token = token.lower()
                if not is_ext_vocab: #trainning set
                    token_id = vocab[token]
                elif token not in vocab:
                    token_id = vocab['<OOV>']
                else:
                    token_id = vocab[token]                 
                sent.append(token_id)
            if len(sent) > max_sent_len:
                max_sent_len = len(sent)
            sents.append(sent)
        data_sentences.append(sents)
        data_orders.append(instance['order'])

    if max_sent_len_ is not None:
        max_sent_len = max_sent_len_
    out_sentences = np.full([len(data_sentences), 5, max_sent_len], vocab['<PAD>'], dtype=np.int32)

    for i, elem in enumerate(data_sentences):
        for j, sent in enumerate(elem):
            out_sentences[i, j, 0:len(sent)] = sent

    out_orders = np.array(data_orders, dtype=np.int32)

    return out_sentences, out_orders, vocab, embd

In [7]:
train_stories, train_orders, vocab, embd = pipeline2(data_train)

Successfully load.


In [8]:
train_stories.shape

(45502, 5, 20)

In [9]:
max_sent_len = train_stories.shape[2]
dev_stories, dev_orders, _, _ = pipeline2(data_dev, vocab=vocab, max_sent_len_=max_sent_len)

In [10]:
train_stories = np.reshape(train_stories,(-1,20))
train_orders = np.reshape(train_orders,(-1,1))
dev_stories = np.reshape(dev_stories,(-1,20))
dev_orders = np.reshape(dev_orders,(-1,1))

In [11]:
train_stories.shape

(227510, 20)

In [43]:
embeddings = tf.convert_to_tensor(embd)

In [44]:
embeddings

<tf.Tensor 'Const_1:0' shape=(26516, 50) dtype=float32>

LSTM MODEL


In [11]:
from tensorflow.contrib import rnn

In [12]:
#training parameter
num_hidden = 128
target_size = 5
vocab_size = len(vocab)
input_size = 50
timesteps = 5
num_layers = 2

In [28]:
tf.reset_default_graph()
#creat tensor holder
story = tf.placeholder(tf.int32, [None,None,None],"story")
order = tf.placeholder(tf.int32, [None,None],"order")
embedding = tf.placeholder(tf.float32,[vocab_size,input_size],"embed")

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, target_size]),dtype=tf.float32)
}
biases = {
    'out': tf.Variable(tf.random_normal([target_size]),dtype=tf.float32)
}

#embedding word
batch_size = tf.shape(story)[0]
sentences = [tf.reshape(x, [batch_size, -1]) for x in tf.split(axis=1, num_or_size_splits=5, value=story)]  # 5 times [batch_size x max_length]
# initializer = tf.random_uniform_initializer(-0.1, 0.1)
# embeddings = tf.get_variable("W", [vocab_size, input_size], initializer=initializer)
embeddings = tf.Variable(tf.constant(0.0,shape = [vocab_size,input_size]),trainable = False)#false means that we don't update the word vector
embedding_init = embeddings.assign(embedding)
sentences_embedded = [tf.nn.embedding_lookup(embeddings, sentence)   # [batch_size x max_seq_length x input_size]
                    for sentence in sentences]
hs = [tf.reduce_sum(sentence, 1) for sentence in sentences_embedded] # 5 times [batch_size x input_size]
# hs = tf.reshape(hs,[5,batch_size,input_size])

#build multiRNNCell
cells = []
for _ in range(num_layers):
    cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
#     cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.5)
    cells.append(cell)
cell = tf.contrib.rnn.MultiRNNCell(cells)
output, states = rnn.static_rnn(cell,hs,dtype=tf.float32)

#basiclstmcell
# lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
# outputs, states = rnn.static_rnn(lstm_cell,hs,dtype=tf.float32)


logits = []
for eachoutput in output:
    logits.append(tf.matmul(eachoutput, weights['out']) + biases['out'])
logits = tf.concat(axis=1, values=logits)#[batch_size x 5*input_size]
logits = tf.reshape(logits, [-1, 5, target_size]) #[batch,5,target]

# calculate loss 
loss = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=order))

# prediction function
unpacked_logits = [tensor for tensor in tf.unstack(logits, axis=1)]
softmaxes = [tf.nn.softmax(tensor) for tensor in unpacked_logits]
softmaxed_logits = tf.stack(softmaxes, axis=1)
predict = tf.arg_max(softmaxed_logits, 2)
opt_op = tf.train.AdamOptimizer(0.1).minimize(loss)


In [29]:
BATCH_SIZE = 25

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    n = train_stories.shape[0]                
    for epoch in range(10000):
        print('----- Epoch', epoch, '-----')
        total_loss = 0
        for i in range(n // BATCH_SIZE):
            inst_story = train_stories[i * BATCH_SIZE: (i + 1) * BATCH_SIZE]
            inst_order = train_orders[i * BATCH_SIZE: (i + 1) * BATCH_SIZE]
            feed_dict = {story: inst_story, order: inst_order,embedding : embd}
            _, current_loss = sess.run([opt_op, loss], feed_dict=feed_dict)
            total_loss += current_loss
        print(' Train loss:', total_loss / n)
        

        train_feed_dict = {story: train_stories, order: train_orders,embedding:embd}
        train_predicted = sess.run(predict, feed_dict=train_feed_dict)
        train_accuracy = nn.calculate_accuracy(train_orders, train_predicted)
        print(' Train accuracy:', train_accuracy)
        
        dev_feed_dict = {story: dev_stories, order: dev_orders,embedding: embd}
        dev_predicted = sess.run(predict, feed_dict=dev_feed_dict)
        dev_accuracy = nn.calculate_accuracy(dev_orders, dev_predicted)
        print(' Dev accuracy:', dev_accuracy)

        
    
    nn.save_model(sess)

----- Epoch 0 -----


ValueError: Cannot feed value of shape (25, 20) for Tensor 'story:0', which has shape '(?, ?, ?)'